# Sample - retrieving DC using OAI-PMH

In [1]:
import requests

In [3]:
rencen_item_url_ex = 'https://cdm17409.contentdm.oclc.org/digital/collection/rencen/id/315/rec/3'
collection_url = 'https://cdm17409.contentdm.oclc.org/digital'
item_set = 'rencen'
item_id = '3'
oai_endpoint = 'https://cdm17409.contentdm.oclc.org/oai/oai.php'

### Identify the repository

In [12]:
parameters = {
    'verb':'Identify'
}

In [13]:
r = requests.get(oai_endpoint, params=parameters)
r.url

'https://cdm17409.contentdm.oclc.org/oai/oai.php?verb=Identify'

In [15]:
print(r.text)

<?xml version="1.0" encoding="UTF-8"?><OAI-PMH xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/ http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd"><responseDate>2024-10-09T21:03:31Z</responseDate><request verb="Identify">http://cdm17409.contentdm.oclc.org/oai/oai.php</request><Identify>
      <repositoryName>CONTENTdm Server Repository</repositoryName>
      <baseURL>http://cdm17409.contentdm.oclc.org/oai/oai.php</baseURL>
      <protocolVersion>2.0</protocolVersion>
      <adminEmail>digitalcollections@wayne.edu</adminEmail>
      <earliestDatestamp>2022-05-04</earliestDatestamp>
      <deletedRecord>transient</deletedRecord>
      <granularity>YYYY-MM-DD</granularity>
   </Identify>
  </OAI-PMH>


### Get a specific record

https://cdm17409.contentdm.oclc.org/oai/oai.php?verb=GetRecord&identifier=oai:cdm17409.contentdm.oclc.org:rencen/3&metadataPrefix=oai_dc

In [16]:
parameters = {
    'verb':'GetRecord',
    'identifier':'oai:cdm17409.contentdm.oclc.org:rencen/3',
    'metadataPrefix':'oai_dc'
}

In [17]:
r = requests.get(oai_endpoint, params=parameters)
r.url

'https://cdm17409.contentdm.oclc.org/oai/oai.php?verb=GetRecord&identifier=oai%3Acdm17409.contentdm.oclc.org%3Arencen%2F3&metadataPrefix=oai_dc'

In [19]:
print(r.text)

<?xml version="1.0" encoding="UTF-8"?><OAI-PMH xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/ http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd"><responseDate>2024-10-09T21:49:43Z</responseDate><request verb="GetRecord" identifier="oai:cdm17409.contentdm.oclc.org:rencen/3" metadataPrefix="oai_dc">http://cdm17409.contentdm.oclc.org/oai/oai.php</request><GetRecord><record><header><identifier>oai:cdm17409.contentdm.oclc.org:rencen/3</identifier><datestamp>2023-03-20</datestamp><setSpec>rencen</setSpec></header>
<metadata>
<oai_dc:dc xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd">
<dc:title>View of the atrium in the completed Renaissance Center.</dc:title>
<dc:descri

#### Parse markup with beautful soup 

In [20]:
from bs4 import BeautifulSoup

In [22]:
dc_example_parsed = BeautifulSoup(r.text, 'xml')

In [23]:
for tag in dc_example_parsed:
    print(tag.name)

OAI-PMH


In [30]:
dc_example_parsed.find_all('dc:coverage')

[<dc:coverage>Detroit, Michigan</dc:coverage>,
 <dc:coverage>1970s</dc:coverage>]

In [34]:
for tag in dc_example_parsed.find('metadata'):
    print(tag)



<oai_dc:dc xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/oai_dc/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd">
<dc:title>View of the atrium in the completed Renaissance Center.</dc:title>
<dc:description>Image of the atrium within the Renaissance Center, looking up at a large concrete structure. Trees, plants, and people can be seen in this picture of a restaurant.</dc:description>
<dc:identifier>rencen_03e</dc:identifier>
<dc:date>1977</dc:date>
<dc:type>still image</dc:type>
<dc:format>photographs</dc:format>
<dc:subject>Renaissance Center (Detroit, Mich.)</dc:subject>
<dc:coverage>Detroit, Michigan</dc:coverage>
<dc:coverage>1970s</dc:coverage>
<dc:relation>Building the Detroit Renaissance Center</dc:relation>
<dc:rights>Users can cite and link to these materials without obtaining permission. Users can also use th